In [1]:
import tensorflow as tf
from keras.layers import Conv2D, UpSampling2D, InputLayer
from keras.models import Sequential
from tensorflow.keras.utils import array_to_img, img_to_array, load_img
import numpy as np
import cv2 as cv
import os
import gc

In [2]:
# Constants
num_samples = 5000
width = 224
height = 224

## Loading images and preprocess

In [3]:
# load all images from a directory
def load_images(path):
    images = []
    x = 0
    for filename in os.listdir(path):
        if x >= num_samples:
            break
        img = cv.imread(os.path.join(path, filename))
        img = cv.resize(img, (width, height))
        if img is not None:
            images.append(img)
        x += 1 
    return images

In [4]:
imgs = load_images('../input/image-colorization/unlabeled2017_subsample')
len(imgs)

5000

In [5]:
# convert to numpy array
imgs = np.array(imgs)

In [6]:
#convert a list of images to lab 
lab_images = np.array([cv.cvtColor(np.float32(1.0/255*imgs[i]), cv.COLOR_BGR2LAB) for i in range(num_samples)])

#free memory
del imgs
gc.collect()

127

In [7]:
#get the L channel
l_images = np.array([np.float32(lab_images[i][:, :, 0]) for i in range(num_samples)])

In [8]:
l_images.shape

(5000, 224, 224)

In [9]:
#get the A and B channels
ab_images = np.array([np.float32(lab_images[i][:, :, 1:]) for i in range(num_samples)])
ab_images /= 128
ab_images.shape
# Free memory
del lab_images
gc.collect()

63

In [10]:
l_images = l_images.reshape(num_samples,width,height,1)
l_images.shape

(5000, 224, 224, 1)

## Model

In [11]:
# Building the neural network
def buildSimpleNN():
    model = Sequential()
    model.add(InputLayer(input_shape=(None, None, 1)))
    model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
    model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))

    model.compile(optimizer='rmsprop', loss='mse')

    return model

## Training model

In [12]:
X = l_images.astype(np.float32)
Y = ab_images.astype(np.float32)

In [13]:
# Training the neural network
def trainModel(X, Y, model):
    print('Training model...')
    model.fit(x=X, y=Y, batch_size=1, epochs=30, verbose=0)
    print('Model trained.')

In [14]:
model = buildSimpleNN()

2022-06-28 03:55:22.952426: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-28 03:55:23.110991: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-28 03:55:23.112085: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-28 03:55:23.114407: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [15]:
trainModel(X, Y, model)

Training model...


2022-06-28 03:55:26.100202: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1003520000 exceeds 10% of free system memory.
2022-06-28 03:55:27.218168: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2007040000 exceeds 10% of free system memory.
2022-06-28 03:55:29.473109: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1003520000 exceeds 10% of free system memory.
2022-06-28 03:55:30.201383: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2007040000 exceeds 10% of free system memory.
2022-06-28 03:55:31.763446: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-06-28 03:55:33.920666: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


Model trained.


In [16]:
from datetime import datetime

# Save model
model.save(datetime.now().strftime("%Y_%m_%d_%H_%M_%S")+'.h5')

## Prediction

In [17]:
# Get the luminance of the image aka the image in black and white
def getLFromLab(image_loaded):
    L_image = cv.cvtColor(np.float32(
        1.0/255 * image_loaded), cv.COLOR_RGB2LAB)[:, :, 0]
    L_image = L_image.reshape(1,width,height, 1)
    return L_image

In [18]:
# Get a image with colorized pixels
def getColorizeImage(image_name, model):
    #Get image data
    image_loaded = img_to_array(load_img(image_name))
    image_loaded = cv.resize(image_loaded, (width, height))
    image_loaded = np.array(image_loaded, dtype=float)
    #Get luminosity/Black&White image
    L_image = getLFromLab(image_loaded)
    #Predict
    output = model.predict(L_image)
    output = output * 128
    #Transform output to image
    cur = np.zeros((width, height, 3))
    cur[:,:,0] = L_image[0][:,:,0]
    cur[:,:,1:] = output[0]
    #Convert to BGR
    cv_result = cv.cvtColor(np.float32(cur),  cv.COLOR_LAB2BGR)
    cv_result = cv_result * 255
    cv_result = cv_result.round(0)
    result_name = 'result_'+datetime.now().strftime("%Y_%m_%d_%H_%M_%S")+'.png'
    cv.imwrite(result_name, cv_result)


In [19]:
getColorizeImage("../input/images/waterfall.jpg", model)